# Obtaining features for a given tested protein

--- 
The notebook provides necessary functions and pipeline to obtain all 14 features used in our study

--- 
## Requirements

- **A folder named as its uniprot ID** 
    - `data/proteins/#uniprotID_of_your_protein#`
    - This folder should include the following three files:
        - `.arff` file (output of our PatMut script)
        - `.cif` file (AlphaFold structure)
        - `.json` file (Arpeggio output with atomic contacts)
   
   
- Additionally, in the QAFI/data folder, you have the following tables: (each file is explained under its title)
    - `feature_neco.csv`
    - `MAX_COUNTS.csv`
    - `MJ_POTENTIAL_TABLE3.csv`
    - `feature_laar.csv`
    
--- 

For a detailed description of the features, please refer to our QAFI paper. Preprint version can be found:
- Selen Ozkan, Natàlia Padilla, Xavier de la Cruz et al. QAFI: A Novel Method for Quantitative Estimation of Missense Variant Impact Using Protein-Specific Predictors and Ensemble Learning, 07 May 2024, PREPRINT (Version 1) available at Research Square [https://doi.org/10.21203/rs.3.rs-4348948/v1]


In [1]:
import pandas as pd
from os.path import isfile, join
import FEATURES
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

<div class="alert alert-info">
  <strong> <h1>Sequence-based Features</h1> </strong>
</div>

Three sequence-based features we use in this study (Blosum62, PSSM, Shannon's entropy) can be obtained using the PatMut pipeline.
- Please follow the instructions provided [here](https://github.com/NataliaSirera/patmut), developed by Natàlia Padilla
- The output of the PatMut pipeline will be an .msa and .arff file

- <b>Note:</b> if you modify the parameters in the `.config` file for changing which features to retrieve, please modify the <b>"parse_arff"</b> function in the FEATURES module (for the "skiprows = ..." and the data.columns = ...)

<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Blosum62 & PSSM & Shannon's entropy</h2></strong>
</div>

Parse PatMut output file (.arff) to extract the three sequence-based features

In [2]:
uniprot = 'Q9Y375'
FEATURES.parse_arff(uniprot)

outputs saved for Q9Y375:

 - data/proteins/Q9Y375/Q9Y375.csv
 - data/proteins/Q9Y375/Q9Y375_entropy.csv


<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Shannon's entropy of sequence neighbors</h2></strong>
</div>

In [3]:
uniprot = 'Q9Y375'
db_protein = pd.read_csv(f'data/proteins/{uniprot}/{uniprot}.csv')

db_protein = FEATURES.add_entropy_window(uniprot, db_protein)

In [4]:
db_protein.head()

,uniprot,variant,first,pos,second,Blosum62,PSSM,Shannon's entropy,Shannon's entropy of seq. neighbours
0,Q9Y375,M1A,M,1,A,-1.0,6.52,0.943,2.051
1,Q9Y375,M1Y,M,1,Y,-1.0,6.52,0.943,2.051
2,Q9Y375,M1W,M,1,W,-1.0,6.52,0.943,2.051
3,Q9Y375,M1V,M,1,V,1.0,6.52,0.943,2.051
4,Q9Y375,M1T,M,1,T,-1.0,6.52,0.943,2.051


<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Neighbor compatibility (neco)</h2></strong>
</div>

In [5]:
uniprot = 'Q9Y375'

db_protein = FEATURES.add_neco(uniprot, db_protein)

In [6]:
db_protein.head()

,uniprot,variant,first,pos,second,Blosum62,PSSM,Shannon's entropy,Shannon's entropy of seq. neighbours,neco
0,Q9Y375,M1A,M,1,A,-1.0,6.52,0.943,2.051,-0.1941
1,Q9Y375,M1Y,M,1,Y,-1.0,6.52,0.943,2.051,-0.5577
2,Q9Y375,M1W,M,1,W,-1.0,6.52,0.943,2.051,-0.4440
3,Q9Y375,M1V,M,1,V,1.0,6.52,0.943,2.051,0.1777
4,Q9Y375,M1T,M,1,T,-1.0,6.52,0.943,2.051,0.1820


In [7]:
db_protein.to_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq.csv', index=0)

<div class="alert alert-info">
  <strong> <h1>Structure-based Features</h1> </strong>
</div>

To calculate the following structural features, we need the following files:

1. AlphaFold structure (.mmCIF file) 

    - The structure of interest can be downloaded from https://alphafold.ebi.ac.uk
    - Jumper, J et al. Highly accurate protein structure prediction with AlphaFold. Nature (2021).
Varadi, M et al. AlphaFold Protein Structure Database in 2024: providing structure coverage for over 214 million protein sequences. Nucleic Acids Research (2024).


2. The atomic interactions within protein structures -> obtained from Arpeggio (.JSON file). 

    - Follow the instructions provided at https://github.com/PDBeurope/arpeggio to generate the necessary JSON file containing all atomic contacts
    - Jubb, Harry C., et al. "Arpeggio: a web server for calculating and visualising interatomic interactions in protein structures." Journal of molecular biology 429.3 (2017): 365-371.

<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>pLDDT and pLDDTbin</h2></strong>
</div>

In [8]:
uniprot = 'Q9Y375'
path = f'data/proteins/{uniprot}/'
db_protein = pd.read_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq.csv')

In [9]:
db_protein2 = FEATURES.add_af_confidencemetric(uniprot, path, db_protein)

Q9Y375 Q9Y375


In [10]:
db_protein2.head()

,uniprot,variant,first,pos,second,Blosum62,PSSM,Shannon's entropy,Shannon's entropy of seq. neighbours,neco,pLDDT,pLDDT bin
0,Q9Y375,M1A,M,1,A,-1.0,6.52,0.943,2.051,-0.1941,38.45,0.0
1,Q9Y375,M1Y,M,1,Y,-1.0,6.52,0.943,2.051,-0.5577,38.45,0.0
2,Q9Y375,M1W,M,1,W,-1.0,6.52,0.943,2.051,-0.4440,38.45,0.0
3,Q9Y375,M1V,M,1,V,1.0,6.52,0.943,2.051,0.1777,38.45,0.0
4,Q9Y375,M1T,M,1,T,-1.0,6.52,0.943,2.051,0.1820,38.45,0.0


<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Contact layer size (colasi)</h2></strong>
</div>

### 1. parse .json file and create atomic contact files

In [11]:
filelist = ['Q9Y375']
path_files = 'data/proteins/'

In [12]:
FEATURES.first_contact_layer_countTables(filelist, path_files)

starting... Q9Y375

No chain filters: ['A']
Interacting entities: ['INTRA_SELECTION']
Contacts: ['vdw' 'proximal' 'vdw_clash' 'CARBONPI' 'DONORPI' 'METSULPHURPI'
 'AMIDEAMIDE' 'AMIDERING'] 

Done!


### 2. Normalization of contacts
The maximum number of interatomic contacts observed for the native residue’s type in a well-curated dataset of 593 PDB structures, list of structures selected from PISCES. For all pdb structures, atomic contacts are obtained, counted and maximum number of contact for each residue is stored in the "maximum_counts.csv" file.

- Wang G, Dunbrack RL (2003) PISCES: A protein sequence culling server. Bioinformatics 19:1589–1591.

In [13]:
# max counts file
max_counts = pd.read_csv('data/MAX_COUNTS.csv')

In [14]:
FEATURES.add_normalized_first_contact_layer(path_files, filelist, max_counts)

-- Adding normalized counts column to "_atomic.csv" file --
Q9Y375
done.


### 3. add colasi feature to protein's dataframe

In [15]:
path_files = 'data/proteins/'

db_protein3 = FEATURES.add_colasi(db_protein2, path_files)

In [16]:
# check there is no na in file for the new feature
FEATURES.which_feature_pos_na(db_protein3,'colasi')

no NaNs in colasi column


In [17]:
db_protein3.head()

,uniprot,variant,first,pos,second,Blosum62,PSSM,Shannon's entropy,Shannon's entropy of seq. neighbours,neco,pLDDT,pLDDT bin,wt_pos,colasi
0,Q9Y375,M1A,M,1,A,-1.0,6.52,0.943,2.051,-0.1941,38.45,0.0,M1,0.006
1,Q9Y375,M1Y,M,1,Y,-1.0,6.52,0.943,2.051,-0.5577,38.45,0.0,M1,0.006
2,Q9Y375,M1W,M,1,W,-1.0,6.52,0.943,2.051,-0.4440,38.45,0.0,M1,0.006
3,Q9Y375,M1V,M,1,V,1.0,6.52,0.943,2.051,0.1777,38.45,0.0,M1,0.006
4,Q9Y375,M1T,M,1,T,-1.0,6.52,0.943,2.051,0.1820,38.45,0.0,M1,0.006


In [18]:
db_protein3.to_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq_str.csv',index=0)

<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Fraction cons. 3D neighbor</h2></strong>
</div>

In [19]:
uniprot = 'Q9Y375'
path_files = 'data/proteins/'

t1 = 4.12*2/3
t2 = 4.12/3

db_protein = pd.read_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq_str.csv')

In [20]:
db_protein_merged = FEATURES.add_fraction_cons_3D_neigh(path_files, db_protein, t1, t2)

Q9Y375


In [21]:
db_protein_merged.head()

,uniprot,variant,first,pos,second,Blosum62,PSSM,Shannon's entropy,Shannon's entropy of seq. neighbours,neco,pLDDT,pLDDT bin,wt_pos,colasi,fraction cons. 3D neighbor
0,Q9Y375,M1A,M,1,A,-1.0,6.52,0.943,2.051,-0.1941,38.45,0.0,M1,0.006,0.0
1,Q9Y375,M1Y,M,1,Y,-1.0,6.52,0.943,2.051,-0.5577,38.45,0.0,M1,0.006,0.0
2,Q9Y375,M1W,M,1,W,-1.0,6.52,0.943,2.051,-0.4440,38.45,0.0,M1,0.006,0.0
3,Q9Y375,M1V,M,1,V,1.0,6.52,0.943,2.051,0.1777,38.45,0.0,M1,0.006,0.0
4,Q9Y375,M1T,M,1,T,-1.0,6.52,0.943,2.051,0.1820,38.45,0.0,M1,0.006,0.0


In [22]:
db_protein_merged.to_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq_str2.csv', index=0)

<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Fraction of accessible and buried neighbors conserved (fanc & fbnc)</h2></strong>
</div>

In [23]:
uniprot = 'Q9Y375'
db_protein = pd.read_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq_str2.csv')
path_files = 'data/proteins/'

In [24]:
db_protein1 = FEATURES.add_exInd_feature(path_files, db_protein)

Q9Y375
exposed/buried binary info added to 
-  "protein dataframe" & "_atomic_norm.csv" files.


In [25]:
db_protein1.head()

,uniprot,variant,first,pos,second,Blosum62,PSSM,Shannon's entropy,Shannon's entropy of seq. neighbours,neco,pLDDT,pLDDT bin,wt_pos,colasi,fraction cons. 3D neighbor,exInd
0,Q9Y375,M1A,M,1,A,-1.0,6.52,0.943,2.051,-0.1941,38.45,0.0,M1,0.006,0.0,1.0
1,Q9Y375,M1Y,M,1,Y,-1.0,6.52,0.943,2.051,-0.5577,38.45,0.0,M1,0.006,0.0,1.0
2,Q9Y375,M1W,M,1,W,-1.0,6.52,0.943,2.051,-0.4440,38.45,0.0,M1,0.006,0.0,1.0
3,Q9Y375,M1V,M,1,V,1.0,6.52,0.943,2.051,0.1777,38.45,0.0,M1,0.006,0.0,1.0
4,Q9Y375,M1T,M,1,T,-1.0,6.52,0.943,2.051,0.1820,38.45,0.0,M1,0.006,0.0,1.0


In [26]:
db_protein2=FEATURES.add_fanc_fbnc(path_files, db_protein1)

Q9Y375
.....median entropy exposed: 1.648
.....median entropy buried: 0.748


In [27]:
db_protein2.to_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq_str3.csv', index=0)

<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Miyazawa-Jernigan potential</h2></strong>
</div>


To calculate the difference in contact energies between the sums of native-neighbour and mutant-neighbour interactions, we used the upper triangle of the 20x20 table provided by Miyazawa and Jernigan (Miyazawa and Jernigan 1996).

- Miyazawa S, Jernigan RL (1996) Residue-Residue Potentials with a Favorable Contact Pair Term and an Unfavorable High Packing Density Term, for Simulation and Threading. J Mol Biol 256:623–644.

In [28]:
uniprot = 'Q9Y375'
path_files = 'data/proteins/'

mj_table = pd.read_csv('data/MJ_POTENTIAL_TABLE3.csv', index_col=0)
mj_table

,C,M,F,I,L,V,W,Y,A,G,T,S,N,Q,D,E,H,R,K,P
C,-5.44,-4.99,-5.80,-5.50,-5.83,-4.96,-4.95,-4.16,-3.57,-3.16,-3.11,-2.86,-2.59,-2.85,-2.41,-2.27,-3.60,-2.57,-1.95,-3.07
M,NaN,-5.46,-6.56,-6.02,-6.41,-5.32,-5.55,-4.91,-3.94,-3.39,-3.51,-3.03,-2.95,-3.30,-2.57,-2.89,-3.98,-3.12,-2.48,-3.45
F,NaN,NaN,-7.26,-6.84,-7.28,-6.29,-6.16,-5.66,-4.81,-4.13,-4.28,-4.02,-3.75,-4.10,-3.48,-3.56,-4.77,-3.98,-3.36,-4.24
I,NaN,NaN,NaN,-6.54,-7.04,-6.05,-5.78,-5.25,-4.58,-3.78,-4.03,-3.52,-3.24,-3.67,-3.17,-3.27,-4.14,-3.63,-3.01,-3.76
L,NaN,NaN,NaN,NaN,-7.37,-6.48,-6.14,-5.67,-4.91,-4.16,-4.34,-3.92,-3.74,-4.04,-3.40,-3.59,-4.54,-4.03,-3.37,-4.20
V,NaN,NaN,NaN,NaN,NaN,-5.52,-5.18,-4.62,-4.04,-3.38,-3.46,-3.05,-2.83,-3.07,-2.48,-2.67,-3.58,-3.07,-2.49,-3.32
W,NaN,NaN,NaN,NaN,NaN,NaN,-5.06,-4.66,-3.82,-3.42,-3.22,-2.99,-3.07,-3.11,-2.84,-2.99,-3.98,-3.41,-2.69,-3.73
Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.17,-3.36,-3.01,-3.01,-2.78,-2.76,-2.97,-2.76,-2.79,-3.52,-3.16,-2.60,-3.19
A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.72,-2.31,-2.32,-2.01,-1.84,-1.89,-1.70,-1.51,-2.41,-1.83,-1.31,-2.03
G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.24,-2.08,-1.82,-1.74,-1.66,-1.59,-1.22,-2.15,-1.72,-1.15,-1.87


In [29]:
FEATURES.mj_potential_prep(uniprot, path_files)

uniprot:  Q9Y375 //  structure file name:  AF-Q9Y375-F1-model_v4
outputs: created in data/proteins/uni/:
- side_chain_table.csv
avg_side_chain_table.csv
-contact_avg_side_chains.csv


In [30]:
db_protein = pd.read_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq_str3.csv')
contact_table = pd.read_csv(path_files+uniprot+'/'+'contact_avg_side_chains.csv')
contact_table.head()

,residue,wt,pos,wt_pos,contact_residue,contact_wt,contact_pos,distance,pLDDT bin
0,SER,S,81,S81,ASP,D,83,6.101,1.0
1,SER,S,81,S81,LYS,K,84,5.043,1.0
2,PHE,F,82,F82,ALA,A,85,6.257,1.0
3,ASP,D,83,D83,ILE,I,86,6.256,1.0
4,LYS,K,84,K84,ASP,D,88,5.330,1.0


In [31]:
db_merged = FEATURES.add_MJ_potential(db_protein,contact_table,mj_table)

In [32]:
db_merged.to_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq_str4.csv', index=0)

<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Accesibility dependent volume term</h2></strong>
</div>

In [33]:
# max counts file
max_counts = pd.read_csv('data/MAX_COUNTS.csv')

path_files = 'data/proteins/'
uniprot = 'Q9Y375'
db_protein = pd.read_csv(f'data/proteins/{uniprot}/{uniprot}_featuresSeq_str4.csv')

In [34]:
db_protein2 = FEATURES.add_acc_dependent_vol(path_files, db_protein, max_counts)

Q9Y375


<div style="background-color: #e4d8ed; padding: 10px; border-radius: 20px;">
  <strong><h2>Likelihood of the accessibility state for a given amino acid replacement (laar) </h2></strong>
</div>

In [35]:
path_files = 'data/proteins/'

In [36]:
db_protein_final = FEATURES.add_laar(path_files, db_protein2)

Q9Y375


In [37]:
# final protein dataframe with all 14 features

db_protein_final

,uniprot,variant,first,pos,second,Blosum62,PSSM,Shannon's entropy,Shannon's entropy of seq. neighbours,neco,...,pLDDT bin,wt_pos,colasi,fraction cons. 3D neighbor,exInd,fanc,fbnc,M.J. potential,access.dependent vol.,laar
0,Q9Y375,M1A,M,1,A,-1.0,6.520,0.943,2.051,-0.1941,...,0.0,M1,0.006,0.000,1.0,0.000,0.0,0.0,-0.254,0.327763
1,Q9Y375,M1Y,M,1,Y,-1.0,6.520,0.943,2.051,-0.5577,...,0.0,M1,0.006,0.000,1.0,0.000,0.0,0.0,0.438,-0.153484
2,Q9Y375,M1W,M,1,W,-1.0,6.520,0.943,2.051,-0.4440,...,0.0,M1,0.006,0.000,1.0,0.000,0.0,0.0,0.550,0.189655
3,Q9Y375,M1V,M,1,V,1.0,6.520,0.943,2.051,0.1777,...,0.0,M1,0.006,0.000,1.0,0.000,0.0,0.0,-0.207,0.067623
4,Q9Y375,M1T,M,1,T,-1.0,6.520,0.943,2.051,0.1820,...,0.0,M1,0.006,0.000,1.0,0.000,0.0,0.0,-0.142,0.470787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,Q9Y375,K327C,K,327,C,-3.0,3.215,1.787,2.144,-0.5291,...,0.0,K327,0.169,0.419,1.0,0.419,0.0,0.0,-8.301,-0.372016
6209,Q9Y375,K327A,K,327,A,-1.0,3.215,1.787,2.144,-0.3896,...,0.0,K327,0.169,0.419,1.0,0.419,0.0,0.0,-9.656,0.116837
6210,Q9Y375,K327W,K,327,W,-3.0,3.215,1.787,2.144,-0.2241,...,0.0,K327,0.169,0.419,1.0,0.419,0.0,0.0,13.383,-0.388762
6211,Q9Y375,K327L,K,327,L,-2.0,3.215,1.787,2.144,-0.3625,...,0.0,K327,0.169,0.419,1.0,0.419,0.0,0.0,-2.880,0.005219


## Adding the protein name as a new column

https://www.uniprot.org/uniprotkb/Q9Y375/entry

In [38]:
db_protein_final['protein'] = 'NDUFAF1'

In [39]:
db_protein_final.columns

Index(['uniprot', 'variant', 'first', 'pos', 'second', 'Blosum62', 'PSSM',
       'Shannon's entropy', 'Shannon's entropy of seq. neighbours', 'neco',
       'pLDDT', 'pLDDT bin', 'wt_pos', 'colasi', 'fraction cons. 3D neighbor',
       'exInd', 'fanc', 'fbnc', 'M.J. potential', 'access.dependent vol.',
       'laar', 'protein'],
      dtype='object')

In [40]:
db_protein_final = db_protein_final[['uniprot', 'protein','variant', 'first', 'pos', 'wt_pos','second',
                                     'Blosum62', 'PSSM',"Shannon's entropy", "Shannon's entropy of seq. neighbours",
                                     'neco','pLDDT', 'pLDDT bin',  'colasi', 'fraction cons. 3D neighbor',
                                     'fanc', 'fbnc', 'M.J. potential', 'access.dependent vol.', 'laar' ]]

In [41]:
uniprot=db_protein_final.uniprot.unique()[0]
db_protein_final.to_csv(f'data/proteins/{uniprot}/{uniprot}_featuresAll.csv', index=0)